### Análise Comparativa: Taxa de Desemprego PNAD Contínua vs PME
================================================================

Este script realiza uma análise comparativa entre as séries de desemprego da 
PNAD Contínua e da Pesquisa Mensal de Emprego (PME)

A PNAD Contínua é a principal pesquisa de força de trabalho do Brasil,
iniciada em 2012. Produz estatísticas mensais sobre desemprego, ocupação
e outras características do mercado de trabalho.

 Fluxo de execução:
 1. Carregamento dos dados (PME e PNAD Contínua)
 2. Análise de correlação e regressão
 3. Ampliação da série PNAD usando modelo
 4. Dessazonalização (opcional)
 5. Geração de visualizações

#### Bibliotecas e configurações

In [1]:
# Carrega bibliotecas
import pandas as pd
import numpy as np
from bcb import sgs
from datetime import datetime
import statsmodels.api as sm
from statsmodels.tsa import x13# Ajuste sazonal
import plotly.graph_objects as go
import os
import warnings

# Configurações gerais
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# Dicionário de cores para visualizações
COLORS = {
    'pnad': '#282f6b',
    'blue': '#282f6b',
    'pme': '#eace3f',
    'yellow': '#eace3f',
    'red': '#b22200',
    'green': '#224f20',
    'purple': '#5f487c',
    'gray': '#666666',
    'orange': '#b35c1e'
}


#### Datas

In [2]:
# Configurações de datas
PME_START_DATE = '2002-03'
PME_END_DATE = '2016-02'
PNAD_SERIES_CODE = 24369

#### CARREGAMENTO E PREPARAÇÃO DOS DADOS

In [3]:
filepath='data/pme.csv'

try:
    # Carrega arquivo CSV com separador ';' e decimal ','
    pme_raw = pd.read_csv(filepath, sep=';', decimal=',')
    
    # Cria range de datas mensais para o período da PME
    start_date = pd.to_datetime(PME_START_DATE)
    end_date = pd.to_datetime(PME_END_DATE)
    date_range = pd.date_range(start=start_date, end=end_date, freq='MS')
    
    # Prepara DataFrame final
    pme_data = pd.DataFrame({
        'Date': date_range,
        'pme': pme_raw['pme'].values
    })
    
    # Define data como índice
    pme_data.set_index('Date', inplace=True)
    
    print(f"✓ Dados PME carregados: {len(pme_data)} observações de {PME_START_DATE} a {PME_END_DATE}")
    
    
except FileNotFoundError:
    print(f"⚠ Arquivo não encontrado: {filepath}")
    pme_data = None
except Exception as e:
    print(f"⚠ Erro ao carregar dados PME: {e}")
    pme_data = None

"""
Carrega dados da PNAD Contínua via API do Banco Central (SGS).
"""
try:
    # Baixa série histórica via API do BCB
    pnad_data = sgs.get({'pnad': PNAD_SERIES_CODE})
    
    print(f"✓ Dados PNAD Contínua carregados: {len(pnad_data)} observações")
    print(f"  Período: {pnad_data.index.min().strftime('%m/%Y')} a {pnad_data.index.max().strftime('%m/%Y')}")
    
        
except Exception as e:
    print(f"⚠ Erro ao carregar dados PNAD: {e}")
    pnad_data = None


"""
Combina as séries de desemprego da PNAD Contínua e PME.
Realiza junção externa (outer join) para preservar todas as observações
de ambas as séries, mesmo nos períodos onde apenas uma está disponível.
- dados_completos: série completa com ambas as pesquisas
- dados_overlap: apenas período de sobreposição (2012-2016)
"""
if pme_data is not None and  pme_data is not None:
    # Junção completa (outer join) - preserva todas as observações
    dados_completos = pnad_data.join(pme_data, how='outer')
    # Junção apenas período de sobreposição (inner join)
    dados_overlap = pnad_data.join(pme_data, how='inner').reset_index()
    print(f" Séries combinadas:")
    print(f" Total de observações: {len(dados_completos)}")
    print(f" Período de sobreposição: {len(dados_overlap)} observações")

else: 
    dados_completos = dados_overlap = None



✓ Dados PME carregados: 168 observações de 2002-03 a 2016-02
✓ Dados PNAD Contínua carregados: 161 observações
  Período: 03/2012 a 07/2025
 Séries combinadas:
 Total de observações: 281
 Período de sobreposição: 48 observações


#### ANÁLISE ESTATÍSTICA

In [4]:
"""
Realiza análise de correlação e regressão linear entre PNAD e PME.

A regressão linear permite estabelecer uma relação quantitativa entre
as duas séries, que será usada para ampliar a série PNAD Contínua
retrospectivamente usando dados da PME.

Parameters
----------
dados_overlap : pandas.DataFrame
    Dados do período de sobreposição entre PNAD e PME
    
Returns
-------
statsmodels.regression.linear_model.RegressionResultsWrapper
    Resultado da regressão linear
"""
print("\n" + "="*60)
print("ANÁLISE DE CORRELAÇÃO E REGRESSÃO")
print("="*60)

# Estatísticas descritivas
print("\nESTATÍSTICAS DESCRITIVAS:")
print(dados_overlap[['pnad', 'pme']].describe())

# Correlação entre as séries
correlation = dados_overlap['pnad'].corr(dados_overlap['pme'])
print(f"\nCorrelação PNAD vs PME: {correlation:.4f}")

# Regressão linear: PNAD = α + β * PME + ε
formula = 'pnad ~ pme'
regression_model = sm.OLS.from_formula(formula, data=dados_overlap).fit()

print("\nRESULTADOS DA REGRESSÃO:")
print(regression_model.summary())



ANÁLISE DE CORRELAÇÃO E REGRESSÃO

ESTATÍSTICAS DESCRITIVAS:
            pnad        pme
count  48.000000  48.000000
mean    7.577083   5.730090
std     0.905360   1.010961
min     6.200000   4.304397
25%     6.900000   4.892217
50%     7.350000   5.443909
75%     8.000000   6.147074
max    10.300000   8.201058

Correlação PNAD vs PME: 0.9615

RESULTADOS DA REGRESSÃO:
                            OLS Regression Results                            
Dep. Variable:                   pnad   R-squared:                       0.924
Model:                            OLS   Adj. R-squared:                  0.923
Method:                 Least Squares   F-statistic:                     562.5
Date:                Fri, 26 Sep 2025   Prob (F-statistic):           1.95e-27
Time:                        10:39:41   Log-Likelihood:               -0.85687
No. Observations:                  48   AIC:                             5.714
Df Residuals:                      46   BIC:                             9.

In [5]:
"""
Amplia a série PNAD Contínua usando o modelo de regressão e dados da PME.
Para o período anterior a 2012 (quando a PNAD Contínua não existia),
utiliza a equação de regressão para estimar valores da PNAD baseados
nos dados da PME disponíveis.

Parameters
----------
dados_completos : pandas.DataFrame
    Série completa com dados de ambas as pesquisas
regression_model : RegressionResults
    Modelo de regressão ajustado
    
Returns
-------
pandas.DataFrame
    Série PNAD Contínua ampliada para todo o período disponível
"""
print("\n" + "="*60)
print("AMPLIAÇÃO DA SÉRIE PNAD CONTÍNUA")
print("="*60)

# Identifica período onde PNAD não existe mas PME existe
dados_extensao = dados_completos[~dados_completos['pnad'].notna()].copy()

if len(dados_extensao) > 0:
    # Aplica modelo de regressão: PNAD_estimada = intercepto + coef * PME
    intercepto = regression_model.params['Intercept']
    coeficiente = regression_model.params['pme']
    
    dados_extensao['pnad'] = intercepto + coeficiente * dados_extensao['pme']
    
    print(f"✓ Série PNAD ampliada para {len(dados_extensao)} observações adicionais")
    print(f"  Equação: PNAD = {intercepto:.4f} + {coeficiente:.4f} * PME")
    print(f"  R² = {regression_model.rsquared:.4f}")
else:
    print("⚠ Nenhuma observação adicional para ampliar")
    # return dados_completos[['pnad']]

# Combina série original + série ampliada
serie_original = dados_completos[['pnad']].dropna()
serie_ampliada = pd.concat([dados_extensao[['pnad']], serie_original])
serie_ampliada.sort_index(inplace=True)

print(f"✓ Série final: {len(serie_ampliada)} observações")
print(f"  Período: {serie_ampliada.index.min().strftime('%m/%Y')} a {serie_ampliada.index.max().strftime('%m/%Y')}")



AMPLIAÇÃO DA SÉRIE PNAD CONTÍNUA
✓ Série PNAD ampliada para 120 observações adicionais
  Equação: PNAD = 2.6433 + 0.8610 * PME
  R² = 0.9244
✓ Série final: 281 observações
  Período: 03/2002 a 07/2025


#### DESSAZONALIZAÇÃO

In [6]:
"""
Configura o software X-13ARIMA-SEATS para dessazonalização.
O X-13ARIMA-SEATS é o software oficial do U.S. Census Bureau para
ajuste sazonal, amplamente utilizado por institutos estatísticos
internacionais para dessazonalizar séries temporais.
"""
try:
    print("\n" + "="*60)
    print("CONFIGURAÇÃO X-13ARIMA-SEATS")
    print("="*60)
    
    # Download e instalação do X-13 (para ambiente Colab/Linux)
    if not os.path.exists('x13as'):
        print("⬇ Baixando X-13ARIMA-SEATS...")
        os.system("wget -q https://www2.census.gov/software/x-13arima-seats/x13as/unix-linux/program-archives/x13as_ascii-v1-1-b59.tar.gz")
        os.system("tar --gunzip --extract --file=x13as_ascii-v1-1-b59.tar.gz")
        print("✓ X-13ARIMA-SEATS instalado")
    
    # Define caminho como variável de ambiente
    os.environ["X13PATH"] = "x13as"
    print("✓ Variável de ambiente X13PATH configurada")
        
except Exception as e:
    print(f"⚠ Erro na configuração X-13: {e}")


CONFIGURAÇÃO X-13ARIMA-SEATS
✓ Variável de ambiente X13PATH configurada


In [14]:
"""
Aplica dessazonalização à série PNAD Contínua ampliada.
A dessazonalização remove padrões sazonais recorrentes, permitindo
melhor análise das tendências de longo prazo e ciclos econômicos
subjacentes na taxa de desemprego.

Parameters
----------
serie_ampliada : pandas.DataFrame
    Série PNAD Contínua ampliada
    
Returns
-------
pandas.DataFrame
    DataFrame com séries original e dessazonalizada
"""
try:
    print("\n" + "="*60)
    print("DESSAZONALIZAÇÃO DA SÉRIE")
    print("="*60)
    
    # Aplica X-13ARIMA-SEATS
    print("⚙ Executando procedimento de dessazonalização...")
    x13_result = x13.x13_arima_analysis(endog=serie_ampliada['pnad'])
    serie_dessazonalizada = x13_result.seasadj
    
    # Combina séries original e dessazonalizada
    serie_final = pd.DataFrame({
        'Date': serie_ampliada.index,
        'pnad': serie_ampliada['pnad'].values,
        'pnad_ampliada_sa': serie_dessazonalizada.values
    })
    
    print("✓ Dessazonalização concluída")
    print(f"  Método: X-13ARIMA-SEATS")
    print(f"  Observações processadas: {len(resultado)}")
      
except Exception as e:
    print(f"⚠ Erro na dessazonalização: {e}")
    # Retorna série original se dessazonalização falhar
    serie_final = pd.DataFrame({
        'Date': serie_ampliada.index,
        'pnad': serie_ampliada['pnad'].values,
        'pnad_ampliada_sa': serie_ampliada['pnad'].values  # Cópia da original
    })



DESSAZONALIZAÇÃO DA SÉRIE
⚙ Executando procedimento de dessazonalização...
✓ Dessazonalização concluída
  Método: X-13ARIMA-SEATS
  Observações processadas: 281


#####  VISUALIZAÇÃO


In [ ]:
"""
Cria gráfico comparativo entre PNAD Contínua e PME.
"""
title="Taxa de Desemprego: PNAD Contínua vs. PME"

# Converte para formato long
dados_long = dados_overlap.melt(id_vars=['Date'], var_name='variable', value_name='value')

fig = go.Figure()

# Adiciona uma série para cada variável
for var in dados_long['variable'].unique():
    df_var = dados_long[dados_long['variable'] == var]
    
    fig.add_trace(go.Scatter(
        x=df_var['Date'],
        y=df_var['value'],
        mode='lines+markers',
        name=var.upper(),
        line=dict(color=COLORS.get(var, '#000000'), width=1.5),
        marker=dict(size=6),
        hovertemplate='<b>%{fullData.name}</b><br>' +
                    'Data: %{x|%b/%Y}<br>' +
                    'Taxa: %{y:.1f}%<extra></extra>'
    ))

# Layout do gráfico
fig.update_layout(
    title=title,
    xaxis_title="",
    yaxis_title="% População Economicamente Ativa",
    legend_title="",
    legend=dict(orientation='h', y=-0.15),
    template="plotly_white",
    height=300,
    width=600,
    margin=dict(l=40, r=40, t=60, b=80),
    annotations=[
        dict(
            text="Fonte: IBGE e BCB",
            x=1, y=-0.25, xref='paper', yref='paper',
            showarrow=False, font=dict(size=10), xanchor='right'
        )
    ]
)

fig.show()



In [20]:
"""
    Cria gráfico da série PNAD Contínua ampliada (com ou sem dessazonalização).
"""
title="Taxa de Desemprego PNAD Contínua (série ampliada)"
fig = go.Figure()

# Identifica colunas de dados (excluindo 'Date')
data_columns = [col for col in serie_final.columns if col != 'Date']

for col in data_columns:
    # Define nome e cor da série
    if col == 'pnad':
        name = 'PNAD Contínua (original)'
        color = COLORS['blue']
    elif col == 'pnad_ampliada_sa':
        name = 'PNAD Contínua (dessazonalizada)'
        color = COLORS['red']
    else:
        name = col
        color = COLORS.get(col, '#666666')
    
    fig.add_trace(go.Scatter(
        x=serie_final['Date'],
        y=serie_final[col],
        mode='lines+markers',
        name=name,
        line=dict(color=color, width=1.5),
        marker=dict(color=color, size=6),
        hovertemplate='<b>%{fullData.name}</b><br>' +
                        'Data: %{x|%b/%Y}<br>' +
                        'Taxa: %{y:.1f}%<extra></extra>'
    ))

# Layout do gráfico
fig.update_layout(
    title=title,
    xaxis_title="",
    yaxis_title="% População Economicamente Ativa",
    legend_title="",
    template="plotly_white",
    height=400,
    width=800,
    margin=dict(l=40, r=40, t=60, b=80),
    xaxis=dict(
        tickformat="%b/%Y",
        dtick="M24"  # Tick a cada 2 anos
    ),
    legend=dict(orientation="h", y=-0.15, x=0.5, xanchor='center'),
    annotations=[
        dict(
            text="Elaboração própria | Fonte: IBGE e BCB",
            x=1.2, y=-0.5, xref='paper', yref='paper',
            showarrow=False, font=dict(size=10), xanchor='right'
        )
    ]
)

fig.show()

In [10]:
"""
Cria gráfico de validação do modelo de regressão.
Compara valores observados da PNAD com valores preditos pelo modelo
baseado nos dados da PME, permitindo avaliar a qualidade do ajuste.

Parameters
----------
dados_overlap : pandas.DataFrame
    Dados do período de sobreposição
regression_model : RegressionResults
    Modelo de regressão ajustado
"""
# Prepara dados para visualização
dados_modelo = pd.DataFrame({
    'Date': dados_overlap['Date'],
    'PNAD Contínua (observado)': dados_overlap['pnad'],
    'PNAD Contínua (predito)': regression_model.fittedvalues
})

# Converte para formato long
dados_long = dados_modelo.melt(id_vars=['Date'])

fig = go.Figure()

for var in dados_long['variable'].unique():
    df_var = dados_long[dados_long['variable'] == var]
    
    # Define cor baseada no tipo
    if 'observado' in var:
        color = COLORS['blue']
    else:
        color = COLORS['red']
    
    fig.add_trace(go.Scatter(
        x=df_var['Date'],
        y=df_var['value'],
        mode='lines+markers',
        name=var,
        line=dict(color=color, width=1.5),
        marker=dict(size=6),
        hovertemplate='<b>%{fullData.name}</b><br>' +
                        'Data: %{x|%b/%Y}<br>' +
                        'Taxa: %{y:.1f}%<extra></extra>'
    ))

# Layout
fig.update_layout(
    title=f"Validação do Modelo de Regressão (R² = {regression_model.rsquared:.3f})",
    xaxis_title="",
    yaxis_title="% População Economicamente Ativa",
    legend_title="",
    legend=dict(orientation='h', y=-0.15),
    template="plotly_white",
    height=300,
    width=600,
    margin=dict(l=40, r=40, t=60, b=80),
    annotations=[
        dict(
            text="Elaboração própria | Fonte: IBGE e BCB",
            x=1, y=-0.25, xref='paper', yref='paper',
            showarrow=False, font=dict(size=10), xanchor='right'
        )
    ]
)

fig.show()

In [22]:

# Sumário final
print("\n" + "="*80)
print("SUMÁRIO DA ANÁLISE")
print("="*80)
print(f"• Período PNAD Contínua original: {pnad_data.index.min().strftime('%m/%Y')} a {pnad_data.index.max().strftime('%m/%Y')}")
print(f"• Período PME: {PME_START_DATE} a {PME_END_DATE}")
print(f"• Correlação PNAD-PME: {dados_overlap['pnad'].corr(dados_overlap['pme']):.4f}")
print(f"• R² do modelo: {regression_model.rsquared:.4f}")
print(f"• Série ampliada final: {len(serie_final)} observações")



SUMÁRIO DA ANÁLISE
• Período PNAD Contínua original: 03/2012 a 07/2025
• Período PME: 2002-03 a 2016-02
• Correlação PNAD-PME: 0.9615
• R² do modelo: 0.9244
• Série ampliada final: 281 observações


In [ ]:
import os
import glob

for f in glob.glob("x13as*gz"):
    os.remove(f)

In [12]:
# -*- coding: utf-8 -*-
"""
Análise Comparativa: Taxa de Desemprego PNAD Contínua vs PME
================================================================

Este script realiza uma análise comparativa entre as séries de desemprego da 
PNAD Contínua e da Pesquisa Mensal de Emprego (PME), incluindo:
- Importação e preparação dos dados
- Análise de correlação e regressão linear
- Ampliação da série PNAD Contínua usando dados da PME
- Dessazonalização das séries
- Visualizações interativas com Plotly

Autor: Análise adaptada do notebook original
Data: 2025
"""

# =============================================================================
# IMPORTAÇÃO DE BIBLIOTECAS
# =============================================================================

import pandas as pd
import numpy as np
from bcb import sgs
from datetime import datetime
import statsmodels.api as sm
from statsmodels.tsa import x13
import plotly.graph_objects as go
import os
import warnings

# Configurações gerais
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# =============================================================================
# CONFIGURAÇÕES E CONSTANTES
# =============================================================================

# Dicionário de cores para visualizações
COLORS = {
    'blue': '#282f6b',
    'yellow': '#eace3f',
    'red': '#b22200',
    'green': '#224f20',
    'purple': '#5f487c',
    'gray': '#666666',
    'orange': '#b35c1e'
}

# Configurações de datas
PME_START_DATE = '2002-03'
PME_END_DATE = '2016-02'
PNAD_SERIES_CODE = 24369

# =============================================================================
# FUNÇÕES DE CARREGAMENTO E PREPARAÇÃO DOS DADOS
# =============================================================================

def load_pme_data(filepath='data/pme.csv'):
    """
    Carrega e prepara os dados da Pesquisa Mensal de Emprego (PME).
    
    A PME foi realizada pelo IBGE de 2002 a 2016 e cobria as principais 
    regiões metropolitanas do Brasil. A série foi descontinuada com o 
    início da PNAD Contínua.
    
    Parameters
    ----------
    filepath : str
        Caminho para o arquivo CSV com dados da PME
        
    Returns
    -------
    pandas.DataFrame
        DataFrame com índice de data e coluna 'pme' com taxa de desemprego
    """
    try:
        # Carrega arquivo CSV com separador ';' e decimal ','
        pme_raw = pd.read_csv(filepath, sep=';', decimal=',')
        
        # Cria range de datas mensais para o período da PME
        start_date = pd.to_datetime(PME_START_DATE)
        end_date = pd.to_datetime(PME_END_DATE)
        date_range = pd.date_range(start=start_date, end=end_date, freq='MS')
        
        # Prepara DataFrame final
        pme_data = pd.DataFrame({
            'Date': date_range,
            'pme': pme_raw['pme'].values
        })
        
        # Define data como índice
        pme_data.set_index('Date', inplace=True)
        
        print(f"✓ Dados PME carregados: {len(pme_data)} observações de {PME_START_DATE} a {PME_END_DATE}")
        return pme_data
        
    except FileNotFoundError:
        print(f"⚠ Arquivo não encontrado: {filepath}")
        return None
    except Exception as e:
        print(f"⚠ Erro ao carregar dados PME: {e}")
        return None


def load_pnad_data():
    """
    Carrega dados da PNAD Contínua via API do Banco Central (SGS).
    
    A PNAD Contínua é a principal pesquisa de força de trabalho do Brasil,
    iniciada em 2012. Produz estatísticas mensais sobre desemprego, ocupação
    e outras características do mercado de trabalho.
    
    Returns
    -------
    pandas.DataFrame
        DataFrame com índice de data e coluna 'pnad' com taxa de desocupação
    """
    try:
        # Baixa série histórica via API do BCB
        pnad_data = sgs.get({'pnad': PNAD_SERIES_CODE})
        
        print(f"✓ Dados PNAD Contínua carregados: {len(pnad_data)} observações")
        print(f"  Período: {pnad_data.index.min().strftime('%m/%Y')} a {pnad_data.index.max().strftime('%m/%Y')}")
        
        return pnad_data
        
    except Exception as e:
        print(f"⚠ Erro ao carregar dados PNAD: {e}")
        return None


def merge_unemployment_series(pnad_data, pme_data):
    """
    Combina as séries de desemprego da PNAD Contínua e PME.
    
    Realiza junção externa (outer join) para preservar todas as observações
    de ambas as séries, mesmo nos períodos onde apenas uma está disponível.
    
    Parameters
    ----------
    pnad_data : pandas.DataFrame
        Dados da PNAD Contínua
    pme_data : pandas.DataFrame
        Dados da PME
        
    Returns
    -------
    tuple
        (dados_completos, dados_overlap): 
        - dados_completos: série completa com ambas as pesquisas
        - dados_overlap: apenas período de sobreposição (2012-2016)
    """
    # Junção completa (outer join) - preserva todas as observações
    dados_completos = pnad_data.join(pme_data, how='outer')
    
    # Junção apenas período de sobreposição (inner join)
    dados_overlap = pnad_data.join(pme_data, how='inner').reset_index()
    
    print(f"✓ Séries combinadas:")
    print(f"  Total de observações: {len(dados_completos)}")
    print(f"  Período de sobreposição: {len(dados_overlap)} observações")
    
    return dados_completos, dados_overlap


# =============================================================================
# FUNÇÕES DE ANÁLISE ESTATÍSTICA
# =============================================================================

def analyze_correlation_and_regression(dados_overlap):
    """
    Realiza análise de correlação e regressão linear entre PNAD e PME.
    
    A regressão linear permite estabelecer uma relação quantitativa entre
    as duas séries, que será usada para ampliar a série PNAD Contínua
    retrospectivamente usando dados da PME.
    
    Parameters
    ----------
    dados_overlap : pandas.DataFrame
        Dados do período de sobreposição entre PNAD e PME
        
    Returns
    -------
    statsmodels.regression.linear_model.RegressionResultsWrapper
        Resultado da regressão linear
    """
    print("\n" + "="*60)
    print("ANÁLISE DE CORRELAÇÃO E REGRESSÃO")
    print("="*60)
    
    # Estatísticas descritivas
    print("\nESTATÍSTICAS DESCRITIVAS:")
    print(dados_overlap[['pnad', 'pme']].describe())
    
    # Correlação entre as séries
    correlation = dados_overlap['pnad'].corr(dados_overlap['pme'])
    print(f"\nCorrelação PNAD vs PME: {correlation:.4f}")
    
    # Regressão linear: PNAD = α + β * PME + ε
    formula = 'pnad ~ pme'
    regression_model = sm.OLS.from_formula(formula, data=dados_overlap).fit()
    
    print("\nRESULTADOS DA REGRESSÃO:")
    print(regression_model.summary())
    
    return regression_model


def extend_pnad_series(dados_completos, regression_model):
    """
    Amplia a série PNAD Contínua usando o modelo de regressão e dados da PME.
    
    Para o período anterior a 2012 (quando a PNAD Contínua não existia),
    utiliza a equação de regressão para estimar valores da PNAD baseados
    nos dados da PME disponíveis.
    
    Parameters
    ----------
    dados_completos : pandas.DataFrame
        Série completa com dados de ambas as pesquisas
    regression_model : RegressionResults
        Modelo de regressão ajustado
        
    Returns
    -------
    pandas.DataFrame
        Série PNAD Contínua ampliada para todo o período disponível
    """
    print("\n" + "="*60)
    print("AMPLIAÇÃO DA SÉRIE PNAD CONTÍNUA")
    print("="*60)
    
    # Identifica período onde PNAD não existe mas PME existe
    dados_extensao = dados_completos[~dados_completos['pnad'].notna()].copy()
    
    if len(dados_extensao) > 0:
        # Aplica modelo de regressão: PNAD_estimada = intercepto + coef * PME
        intercepto = regression_model.params['Intercept']
        coeficiente = regression_model.params['pme']
        
        dados_extensao['pnad'] = intercepto + coeficiente * dados_extensao['pme']
        
        print(f"✓ Série PNAD ampliada para {len(dados_extensao)} observações adicionais")
        print(f"  Equação: PNAD = {intercepto:.4f} + {coeficiente:.4f} * PME")
        print(f"  R² = {regression_model.rsquared:.4f}")
    else:
        print("⚠ Nenhuma observação adicional para ampliar")
        return dados_completos[['pnad']]
    
    # Combina série original + série ampliada
    serie_original = dados_completos[['pnad']].dropna()
    serie_ampliada = pd.concat([dados_extensao[['pnad']], serie_original])
    serie_ampliada.sort_index(inplace=True)
    
    print(f"✓ Série final: {len(serie_ampliada)} observações")
    print(f"  Período: {serie_ampliada.index.min().strftime('%m/%Y')} a {serie_ampliada.index.max().strftime('%m/%Y')}")
    
    return serie_ampliada


# =============================================================================
# FUNÇÕES DE DESSAZONALIZAÇÃO
# =============================================================================

def setup_x13_seasonal_adjustment():
    """
    Configura o software X-13ARIMA-SEATS para dessazonalização.
    
    O X-13ARIMA-SEATS é o software oficial do U.S. Census Bureau para
    ajuste sazonal, amplamente utilizado por institutos estatísticos
    internacionais para dessazonalizar séries temporais.
    
    Returns
    -------
    bool
        True se configuração foi bem-sucedida, False caso contrário
    """
    try:
        print("\n" + "="*60)
        print("CONFIGURAÇÃO X-13ARIMA-SEATS")
        print("="*60)
        
        # Download e instalação do X-13 (para ambiente Colab/Linux)
        if not os.path.exists('x13as'):
            print("⬇ Baixando X-13ARIMA-SEATS...")
            os.system("wget -q https://www2.census.gov/software/x-13arima-seats/x13as/unix-linux/program-archives/x13as_ascii-v1-1-b59.tar.gz")
            os.system("tar --gunzip --extract --file=x13as_ascii-v1-1-b59.tar.gz")
            print("✓ X-13ARIMA-SEATS instalado")
        
        # Define caminho como variável de ambiente
        os.environ["X13PATH"] = "x13as"
        print("✓ Variável de ambiente X13PATH configurada")
        
        return True
        
    except Exception as e:
        print(f"⚠ Erro na configuração X-13: {e}")
        return False


def apply_seasonal_adjustment(serie_ampliada):
    """
    Aplica dessazonalização à série PNAD Contínua ampliada.
    
    A dessazonalização remove padrões sazonais recorrentes, permitindo
    melhor análise das tendências de longo prazo e ciclos econômicos
    subjacentes na taxa de desemprego.
    
    Parameters
    ----------
    serie_ampliada : pandas.DataFrame
        Série PNAD Contínua ampliada
        
    Returns
    -------
    pandas.DataFrame
        DataFrame com séries original e dessazonalizada
    """
    try:
        print("\n" + "="*60)
        print("DESSAZONALIZAÇÃO DA SÉRIE")
        print("="*60)
        
        # Aplica X-13ARIMA-SEATS
        print("⚙ Executando procedimento de dessazonalização...")
        x13_result = x13.x13_arima_analysis(endog=serie_ampliada['pnad'])
        serie_dessazonalizada = x13_result.seasadj
        
        # Combina séries original e dessazonalizada
        resultado = pd.DataFrame({
            'Date': serie_ampliada.index,
            'pnad': serie_ampliada['pnad'].values,
            'pnad_ampliada_sa': serie_dessazonalizada.values
        })
        
        print("✓ Dessazonalização concluída")
        print(f"  Método: X-13ARIMA-SEATS")
        print(f"  Observações processadas: {len(resultado)}")
        
        return resultado
        
    except Exception as e:
        print(f"⚠ Erro na dessazonalização: {e}")
        # Retorna série original se dessazonalização falhar
        return pd.DataFrame({
            'Date': serie_ampliada.index,
            'pnad': serie_ampliada['pnad'].values,
            'pnad_ampliada_sa': serie_ampliada['pnad'].values  # Cópia da original
        })


# =============================================================================
# FUNÇÕES DE VISUALIZAÇÃO
# =============================================================================

def create_comparison_chart(dados_overlap, title="Taxa de Desemprego: PNAD Contínua vs. PME"):
    """
    Cria gráfico comparativo entre PNAD Contínua e PME.
    
    Parameters
    ----------
    dados_overlap : pandas.DataFrame
        Dados do período de sobreposição
    title : str
        Título do gráfico
        
    Returns
    -------
    plotly.graph_objects.Figure
        Figura interativa do Plotly
    """
    # Converte para formato long
    dados_long = dados_overlap.melt(id_vars=['Date'], var_name='variable', value_name='value')
    
    fig = go.Figure()
    
    # Adiciona uma série para cada variável
    for var in dados_long['variable'].unique():
        df_var = dados_long[dados_long['variable'] == var]
        
        fig.add_trace(go.Scatter(
            x=df_var['Date'],
            y=df_var['value'],
            mode='lines+markers',
            name=var.upper(),
            line=dict(color=COLORS.get(var, '#000000'), width=1.5),
            marker=dict(size=6),
            hovertemplate='<b>%{fullData.name}</b><br>' +
                         'Data: %{x|%b/%Y}<br>' +
                         'Taxa: %{y:.1f}%<extra></extra>'
        ))
    
    # Layout do gráfico
    fig.update_layout(
        title=title,
        xaxis_title="",
        yaxis_title="% População Economicamente Ativa",
        legend_title="",
        legend=dict(orientation='h', y=-0.15),
        template="plotly_white",
        height=500,
        width=1000,
        margin=dict(l=40, r=40, t=60, b=80),
        annotations=[
            dict(
                text="Elaboração própria | Fonte: IBGE e BCB",
                x=1, y=-0.25, xref='paper', yref='paper',
                showarrow=False, font=dict(size=10), xanchor='right'
            )
        ]
    )
    
    return fig


def create_extended_series_chart(serie_final, title="Taxa de Desemprego PNAD Contínua (série ampliada)"):
    """
    Cria gráfico da série PNAD Contínua ampliada.
    
    Parameters
    ----------
    serie_final : pandas.DataFrame
        Série PNAD ampliada (com ou sem dessazonalização)
    title : str
        Título do gráfico
        
    Returns
    -------
    plotly.graph_objects.Figure
        Figura interativa do Plotly
    """
    fig = go.Figure()
    
    # Identifica colunas de dados (excluindo 'Date')
    data_columns = [col for col in serie_final.columns if col != 'Date']
    
    for col in data_columns:
        # Define nome e cor da série
        if col == 'pnad':
            name = 'PNAD Contínua (original)'
            color = COLORS['blue']
        elif col == 'pnad_ampliada_sa':
            name = 'PNAD Contínua (dessazonalizada)'
            color = COLORS['red']
        else:
            name = col
            color = COLORS.get(col, '#666666')
        
        fig.add_trace(go.Scatter(
            x=serie_final['Date'],
            y=serie_final[col],
            mode='lines+markers',
            name=name,
            line=dict(color=color, width=1.5),
            marker=dict(color=color, size=6),
            hovertemplate='<b>%{fullData.name}</b><br>' +
                         'Data: %{x|%b/%Y}<br>' +
                         'Taxa: %{y:.1f}%<extra></extra>'
        ))
    
    # Layout do gráfico
    fig.update_layout(
        title=title,
        xaxis_title="",
        yaxis_title="% População Economicamente Ativa",
        legend_title="",
        template="plotly_white",
        height=500,
        width=1000,
        margin=dict(l=40, r=40, t=60, b=80),
        xaxis=dict(
            tickformat="%b/%Y",
            dtick="M24"  # Tick a cada 2 anos
        ),
        legend=dict(orientation="h", y=-0.15, x=0.5, xanchor='center'),
        annotations=[
            dict(
                text="Elaboração própria | Fonte: IBGE e BCB",
                x=1, y=-0.25, xref='paper', yref='paper',
                showarrow=False, font=dict(size=10), xanchor='right'
            )
        ]
    )
    
    return fig


def create_model_validation_chart(dados_overlap, regression_model):
    """
    Cria gráfico de validação do modelo de regressão.
    
    Compara valores observados da PNAD com valores preditos pelo modelo
    baseado nos dados da PME, permitindo avaliar a qualidade do ajuste.
    
    Parameters
    ----------
    dados_overlap : pandas.DataFrame
        Dados do período de sobreposição
    regression_model : RegressionResults
        Modelo de regressão ajustado
        
    Returns
    -------
    plotly.graph_objects.Figure
        Figura interativa do Plotly
    """
    # Prepara dados para visualização
    dados_modelo = pd.DataFrame({
        'Date': dados_overlap['Date'],
        'PNAD Contínua (observado)': dados_overlap['pnad'],
        'PNAD Contínua (predito)': regression_model.fittedvalues
    })
    
    # Converte para formato long
    dados_long = dados_modelo.melt(id_vars=['Date'])
    
    fig = go.Figure()
    
    for var in dados_long['variable'].unique():
        df_var = dados_long[dados_long['variable'] == var]
        
        # Define cor baseada no tipo
        if 'observado' in var:
            color = COLORS['blue']
        else:
            color = COLORS['red']
        
        fig.add_trace(go.Scatter(
            x=df_var['Date'],
            y=df_var['value'],
            mode='lines+markers',
            name=var,
            line=dict(color=color, width=1.5),
            marker=dict(size=6),
            hovertemplate='<b>%{fullData.name}</b><br>' +
                         'Data: %{x|%b/%Y}<br>' +
                         'Taxa: %{y:.1f}%<extra></extra>'
        ))
    
    # Layout
    fig.update_layout(
        title=f"Validação do Modelo de Regressão (R² = {regression_model.rsquared:.3f})",
        xaxis_title="",
        yaxis_title="% População Economicamente Ativa",
        legend_title="",
        legend=dict(orientation='h', y=-0.15),
        template="plotly_white",
        height=500,
        width=1000,
        margin=dict(l=40, r=40, t=60, b=80),
        annotations=[
            dict(
                text="Elaboração própria | Fonte: IBGE e BCB",
                x=1, y=-0.25, xref='paper', yref='paper',
                showarrow=False, font=dict(size=10), xanchor='right'
            )
        ]
    )
    
    return fig


# =============================================================================
# FUNÇÃO PRINCIPAL
# =============================================================================

def main():
    """
    Função principal que executa toda a análise de desemprego.
    
    Fluxo de execução:
    1. Carregamento dos dados (PME e PNAD Contínua)
    2. Análise de correlação e regressão
    3. Ampliação da série PNAD usando modelo
    4. Dessazonalização (opcional)
    5. Geração de visualizações
    """
    print("="*80)
    print("ANÁLISE COMPARATIVA: TAXA DE DESEMPREGO PNAD CONTÍNUA vs PME")
    print("="*80)
    
    # Etapa 1: Carregamento dos dados
    print("\n1. CARREGAMENTO DOS DADOS")
    print("-" * 40)
    
    pme_data = load_pme_data()
    pnad_data = load_pnad_data()
    
    if pme_data is None or pnad_data is None:
        print("⚠ Erro no carregamento dos dados. Encerrando análise.")
        return None
    
    # Etapa 2: Combinação das séries
    print("\n2. COMBINAÇÃO DAS SÉRIES")
    print("-" * 40)
    
    dados_completos, dados_overlap = merge_unemployment_series(pnad_data, pme_data)
    
    # Etapa 3: Análise estatística
    print("\n3. ANÁLISE ESTATÍSTICA")
    print("-" * 40)
    
    regression_model = analyze_correlation_and_regression(dados_overlap)
    
    # Etapa 4: Ampliação da série PNAD
    print("\n4. AMPLIAÇÃO DA SÉRIE PNAD")
    print("-" * 40)
    
    serie_ampliada = extend_pnad_series(dados_completos, regression_model)
    
    # Etapa 5: Dessazonalização (opcional)
    print("\n5. DESSAZONALIZAÇÃO")
    print("-" * 40)
    
    if setup_x13_seasonal_adjustment():
        serie_final = apply_seasonal_adjustment(serie_ampliada)
    else:
        print("⚠ Dessazonalização não disponível. Usando série original.")
        serie_final = pd.DataFrame({
            'Date': serie_ampliada.index,
            'pnad': serie_ampliada['pnad'].values
        })
    
    # Etapa 6: Visualizações
    print("\n6. GERAÇÃO DE VISUALIZAÇÕES")
    print("-" * 40)
    
    # Gráfico 1: Comparação PNAD vs PME (período de sobreposição)
    fig1 = create_comparison_chart(dados_overlap)
    print("✓ Gráfico comparativo PNAD vs PME criado")
    
    # Gráfico 2: Validação do modelo
    fig2 = create_model_validation_chart(dados_overlap, regression_model)
    print("✓ Gráfico de validação do modelo criado")
    
    # Gráfico 3: Série ampliada
    fig3 = create_extended_series_chart(serie_final)
    print("✓ Gráfico da série ampliada criado")
    
    # Etapa 7: Exibição dos resultados
    print("\n7. RESULTADOS")
    print("-" * 40)
    
    # Mostra gráficos
    fig1.show()
    fig2.show()
    fig3.show()
    
    # Sumário final
    print("\n" + "="*80)
    print("SUMÁRIO DA ANÁLISE")
    print("="*80)
    print(f"• Período PNAD Contínua original: {pnad_data.index.min().strftime('%m/%Y')} a {pnad_data.index.max().strftime('%m/%Y')}")
    print(f"• Período PME: {PME_START_DATE} a {PME_END_DATE}")
    print(f"• Correlação PNAD-PME: {dados_overlap['pnad'].corr(dados_overlap['pme']):.4f}")
    print(f"• R² do modelo: {regression_model.rsquared:.4f}")
    print(f"• Série ampliada final: {len(serie_final)} observações")
    
    return {
        'dados_overlap': dados_overlap,
        'serie_ampliada': serie_ampliada,
        'serie_final': serie_final,
        'regression_model': regression_model,
        'figuras': [fig1, fig2, fig3]
    }


# =============================================================================
# EXECUÇÃO
# =============================================================================

if __name__ == "__main__":
    resultados = main()

ANÁLISE COMPARATIVA: TAXA DE DESEMPREGO PNAD CONTÍNUA vs PME

1. CARREGAMENTO DOS DADOS
----------------------------------------
✓ Dados PME carregados: 168 observações de 2002-03 a 2016-02
✓ Dados PNAD Contínua carregados: 161 observações
  Período: 03/2012 a 07/2025

2. COMBINAÇÃO DAS SÉRIES
----------------------------------------
✓ Séries combinadas:
  Total de observações: 281
  Período de sobreposição: 48 observações

3. ANÁLISE ESTATÍSTICA
----------------------------------------

ANÁLISE DE CORRELAÇÃO E REGRESSÃO

ESTATÍSTICAS DESCRITIVAS:
            pnad        pme
count  48.000000  48.000000
mean    7.577083   5.730090
std     0.905360   1.010961
min     6.200000   4.304397
25%     6.900000   4.892217
50%     7.350000   5.443909
75%     8.000000   6.147074
max    10.300000   8.201058

Correlação PNAD vs PME: 0.9615

RESULTADOS DA REGRESSÃO:
                            OLS Regression Results                            
Dep. Variable:                   pnad   R-squared:       


SUMÁRIO DA ANÁLISE
• Período PNAD Contínua original: 03/2012 a 07/2025
• Período PME: 2002-03 a 2016-02
• Correlação PNAD-PME: 0.9615
• R² do modelo: 0.9244
• Série ampliada final: 281 observações
